In [3]:
# Variables setup
date = "2025-2-23"
industry = "AI"
firm = "Nvidia"
tic = "NVDA"

# retrieving historical price of all firms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from openpyxl import load_workbook
import requests
import yfinance as yf
from yahoo_fin.stock_info import get_data
import warnings
warnings.filterwarnings("ignore")

# 1. Stock Info
price = yf.Ticker(tic).info['currentPrice']
share = yf.Ticker(tic).info['sharesOutstanding']

In [4]:
# 1. FRED Risk-free rate
api_key         = ''      # FRED API key
series_id       = ''                                 # Series ID for the 10-year US Treasury bond yield
url             = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json'  # Construct the URL for the FRED API request
response        = requests.get(url)                       # Send the request to the FRED API and store the response
data            = response.json()                         # Convert the response to a JSON object
observation     = data['observations'][-1]                # Extract the most recent observation (i.e., the observation with the latest date)

risk_free_rate  = float(observation['value']) / 100 
discount = risk_free_rate + 0.02 + 1

# 2. Financial Statements 
summary_df = pd.DataFrame(yf.Ticker(tic).financials.T)
summary_df.index.name = 'Date'
summary_df = summary_df[['Total Revenue', 'Cost Of Revenue','Gross Profit', 'Normalized EBITDA','Operating Expense', 
                         'Operating Income', 'EBIT', 'Net Income', 'Diluted EPS']]
balance_df = pd.DataFrame(yf.Ticker(tic).balance_sheet.T)
balance_df.index.name = 'Date'
balance_df = balance_df[['Cash Cash Equivalents And Short Term Investments', "Tangible Book Value", 'Receivables',  'Total Assets',
                          'Total Debt', 'Stockholders Equity', 'Current Assets', 'Current Liabilities','Retained Earnings',
                          'Share Issued', 'Total Capitalization', 'Goodwill And Other Intangible Assets']]
cashflow_df = pd.DataFrame(yf.Ticker(tic).cashflow.T)
cashflow_df.index.name = 'Date'
cashflow_df = cashflow_df[['Free Cash Flow', 'End Cash Position', 'Beginning Cash Position', 'Capital Expenditure', 'Depreciation And Amortization',
                           'Change In Working Capital','Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow']]

financial_df = pd.merge(balance_df, summary_df, on='Date', how='outer')
financial_df = pd.merge(financial_df, cashflow_df,  on='Date', how='outer')
financial_df["Cash"] = financial_df["Cash Cash Equivalents And Short Term Investments"]
financial_df["BV"] = financial_df["Tangible Book Value"]
financial_df["NWC"] = financial_df["Current Assets"] - financial_df["Current Liabilities"] + financial_df["Cash"] 
financial_df["NWC_change"] = financial_df["NWC"] - financial_df["NWC"].shift(1)

In [5]:
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler

def predict_fcf(method='linear', periods=10, lookback=3, epochs=100):
    fcf_data = financial_df["Free Cash Flow"].dropna().astype('float64')
    
    if method == 'linear':
        X = np.arange(len(fcf_data)).reshape(-1, 1)
        model = LinearRegression().fit(X, fcf_data)
        return model.predict(np.arange(len(fcf_data), len(fcf_data) + periods).reshape(-1, 1)).tolist()
    
    elif method == 'arima':
        best_model = auto_arima(fcf_data, seasonal=False, stepwise=True, suppress_warnings=True)
        model_fit = ARIMA(fcf_data, order=best_model.order).fit()
        return model_fit.forecast(steps=periods).tolist()
    
    elif method == 'exponential':
        model = SimpleExpSmoothing(fcf_data).fit(smoothing_level=0.2, optimized=True)
        return model.forecast(periods).tolist()
    
    elif method == 'lstm':
        scaler = StandardScaler()
        fcf_scaled = scaler.fit_transform(np.array(fcf_data).reshape(-1, 1))
        
        X, y = [], []
        for i in range(len(fcf_scaled) - lookback):
            X.append(fcf_scaled[i:i + lookback])
            y.append(fcf_scaled[i + lookback])      
        X, y = np.array(X).reshape(-1, lookback, 1), np.array(y)
        
        model = Sequential([
            LSTM(100, activation='tanh', return_sequences=True, input_shape=(lookback, 1)),
            Dropout(0.2),  
            LSTM(100, activation='tanh'),
            Dropout(0.2), 
            Dense(1)
        ])     
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')
        model.fit(X, y, epochs=epochs, verbose=0, callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)])
        
        future_fcf, last_values = [], list(fcf_scaled[-lookback:])
        for _ in range(periods):
            X_input = np.array(last_values).reshape(1, lookback, 1)
            pred_scaled = model.predict(X_input, verbose=0)[0, 0]
            pred = scaler.inverse_transform([[pred_scaled]])[0, 0]  
            future_fcf.append(pred)
            last_values.append([pred_scaled])
            last_values.pop(0)
        
        return future_fcf
    
    else:
        raise ValueError("Invalid method. Choose from 'linear', 'arima', 'exponential', 'lstm'")

methods = ['linear', 'arima', 'exponential', 'lstm']
predictions = {method: predict_fcf(method) for method in methods}
for method, result in predictions.items():
    print(f'{method.capitalize()} Prediction: {result}')


C:\Users\user\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YE-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\user\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YE-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\user\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YE-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\user\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YE-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\user\anaconda3\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppre

Linear Prediction: [26578000000.000004, 32843700000.000004, 39109400000.000015, 45375100000.000015, 51640800000.000015, 57906500000.000015, 64172200000.000015, 70437900000.00002, 76703600000.00003, 82969300000.00003]
Arima Prediction: [10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0]
Exponential Prediction: [9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0]
Lstm Prediction: [22287139432.224846, 14964913131.048056, 2199913542.4993763, 8223477801.318424, 19812201100.31524, 28843751397.419655, 13063037722.311884, 4139323698.8570147, 4720606375.919956, 21637676201.870102]


In [6]:
def terminal_value(method='growth', TV_growth=0.04, discount_rate=0.1, exit_multiple=None):
    if method == 'growth':
        fcf_data = financial_df["Free Cash Flow"].dropna().astype('float64').values
        return round((fcf_data[-1] * (1 + TV_growth)) / (discount_rate - TV_growth), 1)
    elif method == 'ebitda':
        EBITDA_data = financial_df["Normalized EBITDA"].dropna().astype('float64').values
        return round(EBITDA_data[-1] * exit_multiple, 1)
    else:
        raise ValueError("Invalid method. Choose from 'growth' or 'ebitda'")

growth_TV = terminal_value(method='growth', TV_growth=0.03, discount_rate=0.06)
ebitda_TV = terminal_value(method='ebitda', exit_multiple=9.5)

for method, result in predictions.items():
    print(f'{method.capitalize()} Prediction: {result}')

print(f"Terminal Value of Gordon Growth Model: ${growth_TV}")
print(f"Terminal Value of Exit Multiple Model: ${ebitda_TV}")

Linear Prediction: [26578000000.000004, 32843700000.000004, 39109400000.000015, 45375100000.000015, 51640800000.000015, 57906500000.000015, 64172200000.000015, 70437900000.00002, 76703600000.00003, 82969300000.00003]
Arima Prediction: [10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0, 10913750000.0]
Exponential Prediction: [9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0, 9457704000.0]
Lstm Prediction: [22287139432.224846, 14964913131.048056, 2199913542.4993763, 8223477801.318424, 19812201100.31524, 28843751397.419655, 13063037722.311884, 4139323698.8570147, 4720606375.919956, 21637676201.870102]
Terminal Value of Gordon Growth Model: $927721000000.0
Terminal Value of Exit Multiple Model: $338038500000.0


In [11]:
linear_fcf = predict_fcf(method='linear')
# Intrinsic value
def dcf_EV(fcf_data= linear_fcf, discount=0.1, TV = ebitda_TV,  share = share, periods = 10):
    fcf_data = np.array(fcf_data)
    periods = len(fcf_data)
    discount_factors = (1 + discount) ** np.arange(1, periods + 1)
    pv_fcf = np.sum(fcf_data / discount_factors)
    pv_TV = TV / ((1 + discount) ** periods)
    intrinsic_price = round(pv_fcf + pv_TV, 2)/share
    return intrinsic_price

intrinsic_values = {
    "Linear DCF": dcf_EV() }

print(intrinsic_values, "current price", price)

{'Linear DCF': 17.846837697441174} current price 137.72


## (III). Analysis Report

In [19]:
import os
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

firm = "Nvidia"
class FinancialAnalyst:
    from openai import OpenAI
    def __init__(self, api_key):
        self.api_key = api_key
    
    def ask(self, prompt):
        client = self.OpenAI(api_key=self.api_key)
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "user", "content": prompt}],
          max_tokens=4000,  n=1,  stop=None,  temperature=0.95,top_p= 0.95,
        )
        return response.choices[0].message.content
    
    def generate_response(self, prompt: str) -> str:
        response = self.ask(prompt)
        return response
    # 
    def analyze_part1(self, company_name: str) -> dict:
        part1_prompts = {
            "Business Model": f"tell me about the business model of {firm}",
            "Geographical Segmentation": f"tell me the geographical segmentation of business of {firm}",
            "Product Segment": f"What is the product segment of {firm}" ,           
            "Industry Overview": f"tell me the industry overview of {firm}",
            "SWOT": f"briefly describe the strengths, weaknesses, opportunities and threats on the business model of {firm}",
            "Growth Opportunity": f"What is the growth opportunity for industry of {firm}",            
            "Competitive Analysis": f"give me a competitive analysis of {firm}",
            "Simulation": f"Using 2020 data, can you give me value of brownian motion monte carlo simulation for {firm}",
            "Valuation": f"perform relative valuation of {firm} using previous competitors calculate industry risk for microsoft using previous data"
       }

        part1_results = {}
        for key, prompt in part1_prompts.items():
            part1_results[key] = self.generate_response(prompt)
        return part1_results

    def save_report_to_word(self, company_name: str):
            document = Document()
            document.add_heading('Comprehensive Financial Analysis Report', level=1)
            style = document.styles['Normal']
            font = style.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
            
            document.add_heading('Industrial Analysis', level=2)
            response = self.analyze_part1(company_name)
            for i,j in response.items():
                document.add_heading(i,level=3)
                paragraph = document.add_paragraph(j)
                
            document.save('Financial_Analysis_Report.docx')
            print("Report successfully saved to Financial_Analysis_Report.docx")
    
gpt = FinancialAnalyst(api_key="")
gpt.save_report_to_word("{firm} financial analysis_{date}")

Report successfully saved to Financial_Analysis_Report.docx
